In [4]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from super_resolution_data_loader import *
from SUB_CNN_model import Net

torch.manual_seed(1)
device = torch.device("cuda")

In [5]:
# SubPixelCNN parameters

batch_size = 64
epochs = 500
lr = 0.01
threads = 4
upscale_factor = 4

In [10]:
img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [11]:
print('===> Building model')
model = Net(upscale_factor=upscale_factor).to(device)
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

===> Building model


In [12]:
def train(epoch):
    epoch_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader, 1):
        input_, target = batch[0].to(device), batch[1].to(device)
        
        optimizer.zero_grad()
        loss = criterion(model(input_), target)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        #print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))
    
    scheduler.step() # Decrease learning rate after 100 epochs to 10% of its value
    
    psnr_epoch = 10*log10(1/(epoch_loss / len(training_data_loader)))
    print("===> Epoch {} Complete: Avg. Loss: {:.4f} / PSNR: {:.4f}".format(epoch, epoch_loss / len(training_data_loader), psnr_epoch))

'''
def test():
    avg_psnr = 0
    with torch.no_grad():
        for batch in testing_data_loader:
            input, target = batch[0].to(device), batch[1].to(device)

            prediction = model(input)
            mse = criterion(prediction, target)
            psnr = 10 * log10(1 / mse.item())
            avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))
'''

def checkpoint(epoch):
    model_out_path = "trained_models/super_res_model_epoch_{}.pth".format(epoch)
    #torch.save(model, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

In [13]:
for epoch in range(1, epochs + 1):
    train(epoch)
    #test()    
    if epoch % 30 == 0:
        checkpoint(epoch)

===> Epoch 1 Complete: Avg. Loss: 0.2250 / PSNR: 6.4779
===> Epoch 2 Complete: Avg. Loss: 0.0071 / PSNR: 21.5011
===> Epoch 3 Complete: Avg. Loss: 0.0054 / PSNR: 22.7050
===> Epoch 4 Complete: Avg. Loss: 0.0041 / PSNR: 23.8204
===> Epoch 5 Complete: Avg. Loss: 0.0035 / PSNR: 24.5365
===> Epoch 6 Complete: Avg. Loss: 0.0032 / PSNR: 24.9496
===> Epoch 7 Complete: Avg. Loss: 0.0029 / PSNR: 25.4117
===> Epoch 8 Complete: Avg. Loss: 0.0027 / PSNR: 25.7080


KeyboardInterrupt: 